<a href="https://colab.research.google.com/github/zhaomargot/energyloadforecasts/blob/main/zhaomargot_dso424_goal1_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Margot Zhao

In [ ]:
# install pycaret
!pip install pycaret


# import libraries
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf

# resolution
mpl.rcParams['figure.dpi'] = 300
plt.style.use('seaborn-whitegrid')

In [ ]:
# import data as pandas file
df = pd.read_excel("https://github.com/robertasgabrys/DSO424SPRING2023/blob/main/CompetitionData.xlsx?raw=true")

In [ ]:
# create timestamps
df["Time"] = pd.date_range(start = pd.Timestamp('2002-01-01 00:00'),end = pd.Timestamp('2006-12-31 23:59'),freq='H')
df["Date"] = df["Time"].dt.date
print(df)

In [ ]:
df["Hour"] = df["Time"].dt.strftime("%H")
df["Hour"] = df["Hour"].astype(int)

df["Month"] = [i.month for i in df["Date"]]

df["Year"] = [i.year for i in df["Date"]]

df["Day_of_week"] = [i.isoweekday() for i in df["Date"]]

df["Day_of_year"] = [pd.Period(i, freq='D').day_of_year for i in df["Date"]]

In [ ]:
df2 = df.loc[df["Time"].dt.year < 2006]
print(df2.tail())
df2.set_index("Time", inplace=True)
df2.drop(columns=["Date"], inplace=True)
print(df2)

In [ ]:
df2.reset_index(drop=True, inplace=True)

In [ ]:
from pycaret.regression import *
s = setup(df2, target = "Load", train_size=0.750171116,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              numeric_features = ['Day_of_year', "Tmax", "Tmin", "Year", "Tavg", "Tmed",],
              categorical_features = ["Hour",'Month', 'Day_of_week'], n_features_to_select=2, transformation=True, verbose = False, use_gpu=True, session_id = 123)


In [ ]:
best_model = compare_models(sort = 'MAPE', verbose=False)

In [ ]:
all_results = []
p = pull().iloc[0:1]
all_results.append(p)
print(p)
# MAPE = 0.0656

                                    Model         MAE           MSE  \
lightgbm  Light Gradient Boosting Machine  87535.0448  1.506252e+10   

                 RMSE      R2   RMSLE    MAPE  TT (Sec)  
lightgbm  121794.2421  0.9092  0.0917  0.0656    0.2567  


In [ ]:
best = create_model("lightgbm", fold=3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,93341.3712,17328981610.1401,131639.5898,0.8870,0.0991,0.0711
1,75006.3420,10086205295.1072,100430.1015,0.9296,0.0752,0.0567
2,94257.4214,17772365247.3680,133313.0348,0.9111,0.1008,0.0689
Mean,87535.0448,15062517384.2051,121794.2421,0.9092,0.0917,0.0656
Std,8867.0207,3523436652.7125,15122.1687,0.0174,0.0117,0.0063


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#2005 Forecast
prediction_holdout = predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,95272.7900,16427908496.0189,128171.4028,0.9198,0.0872,0.0639


In [ ]:
#2006 Forecast
final = finalize_model(best)

In [ ]:
print(df)
df.drop(columns=["Load"], inplace=True)
df.reset_index(drop=True, inplace=True)
df2006 = df.drop(df2.index)
df2006.reset_index(drop=True, inplace=True)
print(df2006)

       Tavg  Tmed  Tmax  Tmin       Load                Time        Date  \
0        43  43.0  60.0  31.0  1384494.0 2002-01-01 00:00:00  2002-01-01   
1        42  42.0  58.0  29.0  1392822.0 2002-01-01 01:00:00  2002-01-01   
2        41  41.0  57.0  31.0  1407887.0 2002-01-01 02:00:00  2002-01-01   
3        41  41.0  56.0  30.0  1438658.0 2002-01-01 03:00:00  2002-01-01   
4        40  41.0  53.0  29.0  1484046.0 2002-01-01 04:00:00  2002-01-01   
...     ...   ...   ...   ...        ...                 ...         ...   
43819    71  72.0  77.0  65.0        NaN 2006-12-31 19:00:00  2006-12-31   
43820    70  72.0  75.0  65.0        NaN 2006-12-31 20:00:00  2006-12-31   
43821    70  71.0  75.0  65.0        NaN 2006-12-31 21:00:00  2006-12-31   
43822    69  70.0  74.0  65.0        NaN 2006-12-31 22:00:00  2006-12-31   
43823    69  70.0  73.0  64.0        NaN 2006-12-31 23:00:00  2006-12-31   

       Hour  Month  Year  Day_of_week  Day_of_year  
0         0      1  2002          

In [ ]:
predict_model(final, df2006, verbose=False)

,Tavg,Tmed,Tmax,Tmin,Time,Date,Hour,Month,Year,Day_of_week,Day_of_year,prediction_label
0,60,60.0,68.0,55.0,2006-01-01 00:00:00,2006-01-01,0,1,2006,7,1,8.632645e+05
1,60,60.0,68.0,54.0,2006-01-01 01:00:00,2006-01-01,1,1,2006,7,1,7.867294e+05
2,60,60.0,67.0,54.0,2006-01-01 02:00:00,2006-01-01,2,1,2006,7,1,7.464466e+05
3,59,60.0,68.0,52.0,2006-01-01 03:00:00,2006-01-01,3,1,2006,7,1,7.548888e+05
4,59,61.0,68.0,51.0,2006-01-01 04:00:00,2006-01-01,4,1,2006,7,1,7.721618e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,71,72.0,77.0,65.0,2006-12-31 19:00:00,2006-12-31,19,12,2006,7,365,1.488023e+06
8756,70,72.0,75.0,65.0,2006-12-31 20:00:00,2006-12-31,20,12,2006,7,365,1.447917e+06
8757,70,71.0,75.0,65.0,2006-12-31 21:00:00,2006-12-31,21,12,2006,7,365,1.310218e+06
8758,69,70.0,74.0,65.0,2006-12-31 22:00:00,2006-12-31,22,12,2006,7,365,1.224375e+06
